In [19]:
import tensorflow as tf
import os , numpy as np
os.chdir("D:/PycharmProjects/DS_ML_Self/FSDS_NOV_deepCNNClassifier")

In [20]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_to_model: Path
    training_data: Path
    all_params: dict
    mlfolw_url: str
    params_image_size: list 
    params_batch_size: int
    

In [21]:
from deepClassifier.constant import *
from deepClassifier.utils import read_yaml,save_json, create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_to_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/PetImages",
            mlfolw_url="https://dagshub.com/umaretiya/FSDS_NOV_deepCNNClassifier.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
        )
        return eval_config

In [22]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [23]:
# !pip install mlflow==1.26.1

In [26]:
import tensorflow as tf
from pathlib import Path
import mlflow 
import mlflow.keras
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config:EvaluationConfig):
        self.config = config
    
    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./ 255,
            validation_split=0.30
        )
        
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs,
        )
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_to_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
        
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlfolw_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss":self.score[0], "accuracy":self.score[1]}
            )
            if tracking_url_type_store != "file":
                mlflow.keras.load_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [27]:
try:
    config = ConfigurationManager()
    val_config= config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

Found 7498 images belonging to 2 classes.


InvalidArgumentError: Graph execution error:

Detected at node 'Squeeze' defined at (most recent call last):
    File "d:\PycharmProjects\DS_ML_Self\FSDS_NOV_deepCNNClassifier\env\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "d:\PycharmProjects\DS_ML_Self\FSDS_NOV_deepCNNClassifier\env\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "d:\PycharmProjects\DS_ML_Self\FSDS_NOV_deepCNNClassifier\env\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "d:\PycharmProjects\DS_ML_Self\FSDS_NOV_deepCNNClassifier\env\lib\site-packages\traitlets\config\application.py", line 978, in launch_instance
      app.start()
    File "d:\PycharmProjects\DS_ML_Self\FSDS_NOV_deepCNNClassifier\env\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "d:\PycharmProjects\DS_ML_Self\FSDS_NOV_deepCNNClassifier\env\lib\site-packages\